# QuickStart

_ActionWeaver is an AI application framework that puts function-calling as a first-class feature_.

---

This demonstrates how to seamlessly integrate any Python function into your LLM application using ActionWeaver and either the Azure or OpenAI API.
 

## Patch OpenAI API

In [27]:
from openai import OpenAI
from actionweaver.llms.openai.tools.chat import OpenAIChatCompletion
from actionweaver.llms.openai.tools.tokens import TokenUsageTracker
from actionweaver.llms.openai.tools.tools import Tools
client = OpenAI()
client = OpenAIChatCompletion.patch(client)



In [13]:
from actionweaver import action

@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


@action(name="GetWeather", stop=False)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and paris?"}
  ],
    actions = [get_current_weather],
    # tools = [Tools.from_action_to_json(get_current_weather)]
    stream=True
)



> /Users/tenghu/Code/actiontools/actionweaver/llms/openai/tools/chat.py(272)new_create()
    270 
    271                     # kwargs['messages'] = kwargs['messages'][:2]
--> 272                     api_response = original_create_method(
    273                         *args,
    274                         **kwargs,



ipdb>  c


Getting current weather
> /Users/tenghu/Code/actiontools/actionweaver/llms/openai/tools/chat.py(272)new_create()
    270 
    271                     # kwargs['messages'] = kwargs['messages'][:2]
--> 272                     api_response = original_create_method(
    273                         *args,
    274                         **kwargs,



ipdb>  pp kwargs['messages']


[{'content': 'You are a helpful assistant.', 'role': 'system'},
 {'content': "what's the weather in San Francisco and paris?", 'role': 'user'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_C81BXkr9x9a0OwxjA0NT4fSY', function=Function(arguments='{\n  "location": "San Francisco"\n}', name='GetWeather'), type='function', index=0)]),
 {'content': '{"location": "San Francisco", "temperature": "72", "unit": '
             '"fahrenheit"}',
  'name': 'GetWeather',
  'role': 'tool',
  'tool_call_id': 'call_C81BXkr9x9a0OwxjA0NT4fSY'}]


ipdb>  q


In [30]:
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
    Function
)
# function_call=None, 
messages = [{'content': 'You are a helpful assistant.', 'role': 'system'},
 {'content': "what's the weather in San Francisco and paris?", 'role': 'user'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_C81BXkr9x9a0OwxjA0NT4fSY', function=Function(arguments='{\n  "location": "San Francisco"\n}', name='GetWeather'), type='function', index=0)]),
 {'content': '{"location": "San Francisco", "temperature": "72", "unit": '
             '"fahrenheit"}',
  'name': 'GetWeather',
  'role': 'tool',
  'tool_call_id': 'call_C81BXkr9x9a0OwxjA0NT4fSY'}]


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
    actions = [get_current_weather]
)
# pp kwargs['messages']

> /Users/tenghu/Code/actiontools/actionweaver/llms/openai/tools/chat.py(272)new_create()
    270 
    271                     # kwargs['messages'] = kwargs['messages'][:2]
--> 272                     api_response = original_create_method(
    273                         *args,
    274                         **kwargs,



ipdb>  pp kwargs['messages']


[{'content': 'You are a helpful assistant.', 'role': 'system'},
 {'content': "what's the weather in San Francisco and paris?", 'role': 'user'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_C81BXkr9x9a0OwxjA0NT4fSY', function=Function(arguments='{\n  "location": "San Francisco"\n}', name='GetWeather'), type='function', index=0)]),
 {'content': '{"location": "San Francisco", "temperature": "72", "unit": '
             '"fahrenheit"}',
  'name': 'GetWeather',
  'role': 'tool',
  'tool_call_id': 'call_C81BXkr9x9a0OwxjA0NT4fSY'}]


ipdb>  c


BadRequestError: Error code: 400 - {'error': {'message': "None is not of type 'object' - 'messages.2.function_call'", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [29]:
response

'The current weather in San Francisco is 72°F and in Paris it is 22°C.'

In [ ]:
def print_output(output):
    from collections.abc import Iterable
    if isinstance(output, str):
        print (output)
    elif isinstance(output, Iterable):
        for chunk in output:
            content = chunk.choices[0].delta.content
            if content is not None:
                print(content, end='')


print_output(response)

## Use ActionWeaver and OpenAI API

In [6]:
import os
import json
import openai
from actionweaver.llms.openai.tools.chat import OpenAIChatCompletion
from actionweaver import action

openai.api_key = os.getenv("OPENAI_API_KEY")

@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"

print (json.dumps(get_current_time.get_function_details(), indent=4))

{
    "name": "GetCurrentTime",
    "description": "\n    Use this for getting the current time in the specified time zone.\n    \n    :return: A string representing the current time in the specified time zone.\n    ",
    "parameters": {
        "properties": {},
        "title": "Get_Current_Time",
        "type": "object"
    }
}


In [7]:
chat = OpenAIChatCompletion("gpt-3.5-turbo")
chat.create(messages=[{"role": "user", "content": "what time is it now"}], actions = [get_current_time], stream=True)

Getting current time...


BadRequestError: Error code: 400 - {'error': {'message': "None is not of type 'object' - 'messages.1.function_call'", 'type': 'invalid_request_error', 'param': None, 'code': None}}

## Use ActionWeaver and Azure OpenAI Service

In [8]:
from actionweaver.llms.azure.chat import ChatCompletion

chat = ChatCompletion(
    model="gpt-35-turbo-0613-16k", 
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview",
)

chat.create([{"role": "user", "content": "what time is it"}], actions = [get_current_time], stream=False)

Getting current time...


'The current time is 16:17:35.'

### Force execute an action

In [36]:
get_current_time.invoke(chat, [{"role": "user", "content": "what time"}], stream=False, force=False)

Getting current time...


'The current time is 14:44.'

### Stop the Action in the loop

Every action comes with a stop argument, which is set to False by default, if True this means that the LLM will immediately return the function's output if chosen, but this also restricts the LLM from making multiple function calls. For instance, if asked about the weather in NYC and San Francisco, the model would invoke two separate functions sequentially for each city. However, with `stop=True`, this process is interrupted once the first function returns weather information for either NYC or San Francisco, depending on which city it queries first.

In [8]:
@action(name="GetWeather", stop=False)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [12]:
chat.create([{"role": "user", "content": "What's the weather like in San Francisco, Tokyo?"}], actions = [get_current_weather])

Getting current weather
Getting current weather


'The weather in San Francisco is currently 72 degrees Fahrenheit. In Tokyo, it is 10 degrees Celsius.'

In [13]:
get_current_weather.stop = True

In [14]:
chat.create([{"role": "user", "content": "What's the weather like in San Francisco, Tokyo?"}], actions = [get_current_weather])

Getting current weather


'{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'